## Setup

In [ ]:
import json
import os
import sys
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", '-')
words_by_grade[3][:10]

In [ ]:
RUN_MODEL_EVALS = False # Set this to True if you want to run evaluations. These can be expensive, especially the full evaluation set!

## How to Run Evals

There are three things we want to know for this demo - how to run evals, how to save them, and how to load them.

The evals themselves are stored in `src/evals/eval_list.py`. Currently, these are all made from the GradeSpellingEval class, since that's the only type of evaluation this repo does. This repo takes a list of about 800 words with grades 1-5 from an elementary school spelling list, and tests the model's ability to spell them in various ways.

We have several methods we'll show below, from running a single eval to doing a full evaluation suite.

Data is contained in `src/evals/data`, with files having the format `EVAL_NAME_results_MODEL_NAME.json`. The gpt2 models only store two evals - spelling a word, and detecting the first letter. The 6B parameter GPT-J model contains all the evaluations. This is because even GPT-J struggles with a lot of the evaluations - GPT-2 fails quite badly at them with our current prompts, so we only show evaluations the model can reasonably attempt.

When we upload to WandB, we currently upload the entire `data` folder. If creating a new set of evaluations, I recommend creating a `grade_spelling_eval` folder and a `NEW_EVAL_NAME` folder within `data`.

In [ ]:
# List evals
print([eval.name for eval in EVAL_LIST])

In [ ]:
models = ['gpt2-medium', 'gpt2-large']
evals = [FULL_SPELLING_EVAL, GET_FIRST_LETTER_EVAL]

"""Load existing files"""
with open("../data/grade_spelling_eval_results_gpt2-medium.json") as file:
    eval_results = json.load(file)
    
# Each eval in EVAL_LIST has a name which is referenced in the results.
print(eval_results.keys())

# Format: eval_results[eval_name][number_of_shots]['data'][grade][idx of entries]. 
# 'data' has raw data, 'accuracy' has results.
print(eval_results[FULL_SPELLING_EVAL.name]['2']['data']['3'][0])

# model, tokenizer = load_huggingface_model('gpt2-medium')

"""Get evaluation dictionaries - comment out the ones you wish to see and analyse, since they take some time.
Est. time for GPT-2 medium is around 30-45 secs per eval on an ATX 6000.
Each few-shot parameter counts as one eval, so run_eval_with_multiple_shots adds up quick.
All four evals should take about 10 mins on an ATX 6000"""

#single_shot_eval = FULL_SPELLING_EVAL.run_eval(model, ModelType.HUGGINGFACE, tokenizer, words_by_grade, 2)
#multi_shot_eval = FULL_SPELLING_EVAL.run_eval_with_multiple_shots(model, ModelType.HUGGINGFACE, tokenizer, words_by_grade, [0, 1, 2, 5, 10])
#eval_set = run_evaluation_set('../data/test_eval', 'gpt2-medium', evals, words_by_grade, 2, True, False)
#multi_model_eval_set = run_multiple_model_evaluation_set('../data', models, evals, words_by_grade, 2, True, False)

"""Upload to WandB"""
# create_wandb_artifact('test_project_name', 'test_artifact_name', '../data')

"""Create color-coded table of correct and incorrect spelling."""
table = create_table(eval_results[FULL_SPELLING_EVAL.name]['2']['data']['3'])
table